In [65]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [66]:
# Generate the SMIRNOFF ligand Structure object
ifs = oechem.oemolistream('../OpenMMCubes/tests/input/toluene.pdb')
mol = oechem.OEMol()
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
oechem.OETriposAtomNames(mol)

from smarty.forcefield import ForceField
mol_ff = ForceField('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml')
mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys, xyz=mol_pos) 
molecule_structure.residues[0].name = "MOL"

print(mol_ff)
print(molecule_structure)
print(molecule_structure.topology)
print(molecule_structure.positions)

<Structure 15 atoms; 1 residues; 15 bonds; parametrized>
<Topology; 1 chains, 1 residues, 15 atoms, 15 bonds>
[(22.370000839233398, 21.680000305175781, 29.770000457763672), (22.600000381469727, 22.930000305175781, 30.420000076293945), (23.399999618530273, 21.079999923706055, 29.010000228881836), (23.760000228881836, 23.590000152587891, 30.25), (24.620000839233398, 21.760000228881836, 28.950000762939453), (24.829999923706055, 22.979999542236328, 29.579999923706055), (26.100000381469727, 23.680000305175781, 29.399999618530273), (21.389999389648438, 21.219999313354492, 29.860000610351562), (21.829999923706055, 23.420000076293945, 31.020000457763672), (23.149999618530273, 20.180000305175781, 28.459999084472656), (23.989999771118164, 24.569999694824219, 30.680000305175781), (25.440000534057617, 21.270000457763672, 28.409999847412109), (25.909999847412109, 24.729999542236328, 29.149999618530273), (26.670000076293945, 23.639999389648438, 30.340000152587891), (26.790000915527344, 23.1900005340

In [67]:
with oechem.oemolostream('smirff_mol.oeb.gz') as ofs:
    ffxml = open('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml', 'rb')
    mol.SetData(oechem.OEGetTag('forcefield'), ffxml.read())
    oechem.OEWriteMolecule(ofs, mol)

In [68]:
# Generate the SMIRNOFF ligand Structure object
with oechem.oemolistream('smirff_mol.oeb.gz') as ifs:
    mol = oechem.OEMol()
    oechem.OEReadMolecule(ifs, mol)
    ffxml = mol.GetData(oechem.OEGetTag('forcefield')).encode()
    
    with open('smirff.ffxml', 'wb') as out:
        out.write(ffxml)
        
    from smarty.forcefield import ForceField
    mol_ff = ForceField(open('smirff.ffxml'))
    mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
    print(mol_pos)

[[ 22.37000084  21.68000031  29.77000046]
 [ 22.60000038  22.93000031  30.42000008]
 [ 23.39999962  21.07999992  29.01000023]
 [ 23.76000023  23.59000015  30.25      ]
 [ 24.62000084  21.76000023  28.95000076]
 [ 24.82999992  22.97999954  29.57999992]
 [ 26.10000038  23.68000031  29.39999962]
 [ 21.38999939  21.21999931  29.86000061]
 [ 21.82999992  23.42000008  31.02000046]
 [ 23.14999962  20.18000031  28.45999908]
 [ 23.98999977  24.56999969  30.68000031]
 [ 25.44000053  21.27000046  28.40999985]
 [ 25.90999985  24.72999954  29.14999962]
 [ 26.67000008  23.63999939  30.34000015]
 [ 26.79000092  23.19000053  28.71999931]] A


In [105]:
oebfname = '../simulation.oeb.gz'
systag = oechem.OEGetTag('states')
ifs = oechem.oemolistream(oebfname)
for mol in ifs.GetOEMols():
    data = mol.GetData(systag)
    system = openmm.XmlSerializer.deserialize( data )
    print(type(system))

ValueError: GetData: states not found.

In [92]:
systag = oechem.OEGetTag('system')
sys_check = oechem.OEHasTag('system')
serialized_system =  mol.GetData(systag)
system = openmm.XmlSerializer.deserialize( serialized_system )
print(type(system))

<class 'simtk.openmm.openmm.System'>


In [46]:
oebfname = '../simulation.oeb.gz'
systag = oechem.OEGetTag('system')
statetag = oechem.OEGetTag('state')
logtag = oechem.OEGetTag('log')
ifs = oechem.oemolistream(oebfname)
for mol in ifs.GetOEMols():
    print(mol)
    system = openmm.XmlSerializer.deserialize( mol.GetData(systag) )
    print(type(system))
    
    state = openmm.XmlSerializer.deserialize( mol.GetData(statetag) )
    print(type(state))
    
    log = mol.GetData(logtag)
    print(log)
    

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fa1a64e3f00> >
<class 'simtk.openmm.openmm.System'>
<class 'simtk.openmm.openmm.State'>
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-428542.16782426834,268.27374506936474
2000,-415328.90217246115,292.01381518311763
3000,-410227.29526019096,300.52425729114134
4000,-410178.7364115864,296.1576736337405
5000,-408896.961594522,300.5506400232759
6000,-409723.38068178296,299.9514962746391
7000,-410476.6667842157,299.47896456019174
8000,-407904.6782325804,300.0447198094963
9000,-409309.79629419,299.9292946386496
10000,-409230.8660710901,301.3697347440811



In [69]:
# Generate protein Stucture object
pdbfilename = os.path.join('..','OpenMMCubes', 'tests', 'input', 'T4-protein.pdb')
pdbfile = app.PDBFile(pdbfilename)
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')
system = forcefield.createSystem( pdbfile.topology )
protein_structure = parmed.openmm.load_topology( pdbfile.topology, system)

print(protein_structure)
print(protein_structure.topology)

<Structure 2634 atoms; 164 residues; 2654 bonds; parametrized>
<Topology; 1 chains, 164 residues, 2634 atoms, 2654 bonds>


In [70]:
# Merge structures
structure = protein_structure + molecule_structure
topology = structure.topology

# Concatenate positions arrays
positions_unit = unit.angstroms
positions0_dimensionless = np.array( pdbfile.positions / positions_unit )
positions1_dimensionless = np.array( mol_pos / positions_unit )

coordinates = np.vstack((positions0_dimensionless,positions1_dimensionless))
natoms = len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, positions_unit)

# Store in Structure object
structure.coordinates = positions

# Save to PDB
structure.save('tmp.pdb',overwrite=True)

In [71]:
# Solvate with PDBFixer
fixer = pdbfixer.PDBFixer('tmp.pdb')
#fixer.findMissingResidues()
#fixer.findMissingAtoms()
#fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*unit.angstrom, ionicStrength=0.050*unit.molar)

In [72]:
# Load PDBFixer object back to Structure
struct = parmed.openmm.load_topology(fixer.topology, xyz=fixer.positions)
# Remove from ligand from protein Structure
struct.strip(":MOL")
struct

<Structure 32003 atoms; 9971 residues; 22216 bonds; PBC (orthogonal); NOT parametrized>

In [73]:
# Regenerate OpenMM System to parameterize solvent
system = forcefield.createSystem(struct.topology, rigidWater=False)

# Regenerate parameterized protein structure
protein_structure = parmed.openmm.load_topology( struct.topology, system=system, 
                                                xyz=struct.positions, box=struct.box )

In [74]:
# Remerge with ligand structure
combined_structure = protein_structure + molecule_structure

# Restore initial positions and box dimensions
combined_structure.positions = fixer.positions
combined_structure.box = struct.box
os.remove('tmp.pdb')
combined_structure.save('combined_structure.pdb', overwrite=True)

In [75]:
print(combined_structure.positions)

[(33.429999351501465, 11.28000020980835, 44.48999881744385), (32.60462999343872, 11.493955850601196, 45.22629737854004), (33.153676986694336, 10.200899839401245, 44.29105758666992), (34.3204140663147, 11.087702512741089, 45.15616416931152), (33.55000019073486, 11.970000267028809, 43.20000171661377), (33.95251989364624, 12.980294227600098, 43.37602615356445), (34.54999923706055, 11.390000581741333, 42.270002365112305), (35.49463510513306, 11.113407611846924, 42.80179023742676), (34.14730787277222, 10.408506393432617, 41.907310485839844), (34.730000495910645, 12.319999933242798, 41.020002365112305), (35.600571632385254, 11.879148483276367, 40.510592460632324), (33.85598182678223, 12.65787959098816, 40.43812274932861), (35.269999504089355, 14.049999713897705, 41.30000114440918), (37.04999923706055, 13.799999952316284, 41.76000118255615), (37.00626611709595, 13.958131074905396, 42.866549491882324), (37.70102024078369, 12.894872426986694, 41.77202224731445), (37.4589729309082, 14.6947777271

In [9]:
# Regenerate OpenMM system with ParmEd
system = combined_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms, 
                                constraints=app.HBonds)

In [10]:
pdbfilename = 'combined_structure.pdb'
pdbfile = app.PDBFile(pdbfilename)
topology = pdbfile.getTopology()
positions = pdbfile.getPositions()

In [11]:
integrator = openmm.LangevinIntegrator(310*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator )

In [12]:
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(310*unit.kelvin)
simulation.minimizeEnergy()

In [26]:
statereporter = app.StateDataReporter('output.log', 1000, step=True, potentialEnergy=True, temperature=True)
simulation.reporters.append(statereporter)

In [20]:
print(statereporter)
dir(statereporter)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkForErrors',
 '_constructHeaders',
 '_constructReportValues',
 '_density',
 '_elapsedTime',
 '_hasInitialized',
 '_initializeConstants',
 '_kineticEnergy',
 '_needEnergy',
 '_needsForces',
 '_needsPositions',
 '_needsVelocities',
 '_openedFile',
 '_out',
 '_potentialEnergy',
 '_progress',
 '_remainingTime',
 '_reportInterval',
 '_separator',
 '_speed',
 '_step',
 '_temperature',
 '_time',
 '_totalEnergy',
 '_totalMass',
 '_totalSteps',
 '_volume',
 'describeNextReport',
 'report']

In [27]:
simulation.step(10000)

In [28]:
state = simulation.context.getState( getPositions=True,
                                     getVelocities=True,
                                     getForces=True,
                                     getEnergy=True,
                                     getParameters=True,
                                     enforcePeriodicBox=True )

In [32]:
print(statereporter)

In [34]:
print(type(state))

<class 'simtk.openmm.openmm.State'>


In [35]:
if type(state) is openmm.State:
    print('True')

True
